In [ ]:
! pip install --upgrade transformers==4.50.1 trl
! pip install 'accelerate>=0.26.0'

In [ ]:
import copy
import json
import os
import pathlib
import warnings

from datasets import Dataset, load_dataset
import pandas as pd
from peft import LoraConfig, get_peft_model
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer
from trl import SFTConfig, SFTTrainer
from tqdm import tqdm
from typing import Optional, Callable
import wandb

# Create Dataset

In [4]:
data = pd.read_csv('data/salary_labelled_development_set_cleaned.csv')

In [5]:
data

,job_id,job_title,job_ad_details,nation_short_desc,salary_additional_text,y_true
0,72000415,Financial Account - Call Center Agent - Up to 34k,\n \n Job Opening \n \n \n Financial Account -...,PH,NaN,17500-17500-PHP-MONTHLY
1,69481519,Aspiring Call Center Agents - Work from Home -...,\n\nJob Opening\n\nAspiring Call Center Agents...,PH,NaN,16000-16000-PHP-MONTHLY
2,55838599,Production Staff Required - Afternoon & Night-...,Original Foods Baking Co. is one of New Zealan...,NZ,NaN,0-0-None-None
3,64369104,Payer Analyst,The Payer Analyst individual is assigned to ...,PH,-,0-0-None-None
4,54861511,"Solicitor, Restructuring",The DLA Piper team operates across more than 4...,AUS,NaN,0-0-None-None
...,...,...,...,...,...,...
2248,75640730,"Senior Manager, Process Excellence and Transfo...",Our client is a well-established consumer good...,HK,NaN,0-0-None-None
2249,77931852,Accounting Manager - Hotel Business,Position: Accounting ManagerJob DescriptionCo...,TH,NaN,0-0-None-None
2250,54359262,Technical Operator - Audio Visual,Your mission: To create memorable experiences ...,AUS,NaN,0-0-None-None
2251,73457326,Customer Service Officer,Working Day: 5-6-5-6 work weeks (rotating base...,SG,"$2,500 – $3,000 per month",2500-3000-SGD-MONTHLY


In [15]:
dataset = []

for i in tqdm(range(data.shape[0])):
    job_id = data.iloc[i].job_id
    desc = {
        "job_title": data.iloc[i].job_title,
        "job_ad_details": data.iloc[i].job_ad_details,
        "nation_short_desc": data.iloc[i].nation_short_desc,
        "salary_additional_text": data.iloc[i].salary_additional_text,
    }
    desc_str = str(desc)
    
    min_salary, max_salary, currency, frequency = data.iloc[i].y_true.split('-')
    label = f'{{"MinSalary": "{min_salary}", "MaxSalary": "{max_salary}", "Currency": "{currency}", "Frequency": "{frequency}"}}'
    
    messages = []
    messages.append(
        {
            'role': 'system',
            'content': 'You are an expert job ad annotator. Your task is to extract structured salary information from job descriptions in the format: min-max-currency-frequency. If salary is not found, return: 0-0-None-None.'
        }
    )
    messages.append(
        {
            'role': 'user',
            'content': (
                f"{desc_str} Extract structured salary information from this job description in the format: min-max-currency-frequency. "
                "Respond in JSON: {\"MinSalary\": \"\", \"MaxSalary\": \"\", \"Currency\": \"\", \"Frequency\": \"\"}. "
                "If not provided explicitly, output 0 for \"MinSalary\" and \"MaxSalary\", and \"None\" for \"Currency\" and \"Frequency\". "
                "If the salary is mentioned, always output a range, where MinSalary and MaxSalary can be equal. "
                "Use 'nation_short_desc' to determine the correct currency. "
                "Output the currency as 3 letters. Use adverb to output frequency (annual, monthly, daily or hourly)."
            )
        }
    )
    messages.append(
        {
            'role': 'assistant',
            'content': label
        }
    )
    
    dataset.append({'messages': messages})

100%|██████████| 2253/2253 [00:00<00:00, 2464.00it/s]


In [16]:
dataset[0]

{'messages': [{'role': 'system',
   'content': 'You are an expert job ad annotator. Your task is to extract structured salary information from job descriptions in the format: min-max-currency-frequency. If salary is not found, return: 0-0-None-None.'},
  {'role': 'user',
   'content': '{\'job_title\': \'Financial Account - Call Center Agent - Up to 34k\', \'job_ad_details\': \'\\n \\n Job Opening \\n \\n \\n Financial Account - Call Center Agent - Up to 34k\\n \\n\\n\\n\\n \\n Job Industry\\n \\n \\n \\n Telecommunications \\n\\n\\n\\n \\n Job Type \\n \\n \\n Full-Time \\n\\n\\n\\n \\n Experience Level\\n \\n \\n \\n Entry Level \\n\\n\\n\\n \\n Date Posted \\n \\n \\n 2022-10-27 \\n\\n\\n\\n \\n Job Location \\n \\n \\n Pasig BlvdPasig1000NCRPhilippines \\n\\n\\n\\n \\n Company Information \\n \\n \\n Sapient\\n \\n Pasig Blvd \\n Cebu, Central Visayas \\n 6019 \\n Sapient is Philippine-based BPO that provides a range of outsourcing services from consulting services, IT-enabled servi

In [17]:
with open('salary_dataset.json', 'w') as f:
    json.dump(dataset, f)

# Finetune Qwen2.5 1.5B

In [5]:
dataset = load_dataset("json", data_files="salary_dataset.json")

Generating train split: 0 examples [00:00, ? examples/s]

In [6]:
dataset

DatasetDict({
    train: Dataset({
        features: ['messages'],
        num_rows: 2253
    })
})

In [7]:
model_name = "Qwen/Qwen2.5-1.5B-Instruct"

In [8]:
# Load model and tokenizer.
model = AutoModelForCausalLM.from_pretrained(model_name, device_map="auto")
tokenizer = AutoTokenizer.from_pretrained(
   model_name, padding=True, truncation=True
)

config.json:   0%|          | 0.00/660 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/3.09G [00:00<?, ?B/s]

Sliding Window Attention is enabled but not implemented for `sdpa`; unexpected results may be encountered.


generation_config.json:   0%|          | 0.00/242 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/7.30k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/2.78M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/1.67M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/7.03M [00:00<?, ?B/s]

In [9]:
wandb.login()

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:

  ········


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


True

In [10]:
seed = 123

num_train_epochs = 5
max_steps = -1
bf16 = False
output_dir = 'finetune_qwen_results'
run_name = f"{model_name.split('/')[-1]}-salary"
output_dir_final = os.path.join(output_dir, run_name)
pathlib.Path(output_dir_final).mkdir(parents=True, exist_ok=True)

# Adjust tokenizer settings as warned by the trainer
tokenizer.padding_side = 'right'

print("Creating trainer...")
training_args = SFTConfig(
    num_train_epochs=num_train_epochs,
    per_device_train_batch_size=1,
    gradient_accumulation_steps=8,
    gradient_checkpointing=True,
    bf16=bf16,
    tf32=False, # use tf32 for faster training on Ampere GPUs or newer.
    dataloader_pin_memory=False,
    torch_compile=False,
    warmup_steps=50,
    max_steps=max_steps,
    learning_rate=1e-4,
    lr_scheduler_type="cosine",
    weight_decay=0.01,
    logging_strategy="steps",
    save_strategy="steps",
    save_steps=1500,
    save_total_limit=10,
    logging_steps=50,
    output_dir=output_dir_final,
    optim="paged_adamw_8bit",
    remove_unused_columns=True,
    seed=seed,
    run_name=run_name,
    report_to="wandb",
    push_to_hub=False,
)

trainer = SFTTrainer(
    model=model,
    args=training_args,
    train_dataset=dataset['train'],
    processing_class=tokenizer,
)

print("Training...")
trainer.train()

Creating trainer...


Converting train dataset to ChatML:   0%|          | 0/2253 [00:00<?, ? examples/s]

Applying chat template to train dataset:   0%|          | 0/2253 [00:00<?, ? examples/s]

Tokenizing train dataset:   0%|          | 0/2253 [00:00<?, ? examples/s]

Truncating train dataset:   0%|          | 0/2253 [00:00<?, ? examples/s]

...

`use_cache=True` is incompatible with gradient checkpointing. Setting `use_cache=False`.


Step,Training Loss
50,1.741500
100,1.628100
150,1.583500
200,1.518400
250,1.521600
300,1.328300
350,0.945400
400,1.024000
450,1.036800
500,0.962700


TrainOutput(global_step=1405, training_loss=0.6399988450083444, metrics={'train_runtime': 14897.7941, 'train_samples_per_second': 0.756, 'train_steps_per_second': 0.094, 'total_flos': 6.089261301578342e+16, 'train_loss': 0.6399988450083444})

Error in callback <bound method _WandbInit._pause_backend of <wandb.sdk.wandb_init._WandbInit object at 0x7fbf2cc8c550>> (for post_run_cell), with arguments args (<ExecutionResult object at 7fc1300fd690, execution_count=10 error_before_exec=None error_in_exec=None info=<ExecutionInfo object at 7fbf2ffda2d0, raw_cell="seed = 123

num_train_epochs = 5
max_steps = -1
bf.." store_history=True silent=False shell_futures=True cell_id=7b8e63c6-b5e3-4e36-a372-2a0ce14b75fa> result=TrainOutput(global_step=1405, training_loss=0.6399988450083444, metrics={'train_runtime': 14897.7941, 'train_samples_per_second': 0.756, 'train_steps_per_second': 0.094, 'total_flos': 6.089261301578342e+16, 'train_loss': 0.6399988450083444})>,),kwargs {}:


TypeError: _WandbInit._pause_backend() takes 1 positional argument but 2 were given

In [11]:
wandb.finish()

Error in callback <bound method _WandbInit._resume_backend of <wandb.sdk.wandb_init._WandbInit object at 0x7fbf2cc8c550>> (for pre_run_cell), with arguments args (<ExecutionInfo object at 7fbf2c0f0550, raw_cell="wandb.finish()" store_history=True silent=False shell_futures=True cell_id=4e3cb0f3-2e57-4c1e-b94b-5c95d7641424>,),kwargs {}:


TypeError: _WandbInit._resume_backend() takes 1 positional argument but 2 were given

train/epoch,▁▁▂▂▂▂▃▃▃▃▄▄▄▄▅▅▅▅▆▆▆▆▇▇▇▇███
train/global_step,▁▁▂▂▂▂▃▃▃▃▄▄▄▄▅▅▅▅▆▆▆▆▇▇▇▇███
train/grad_norm,█▆▆▅▅▅▅▅▄▅▄▅▄▅▆▄▂▃▃▃▄▃▁▂▁▁▂▁
train/learning_rate,█████▇▇▇▇▆▆▆▅▅▄▄▄▃▃▂▂▂▂▁▁▁▁▁
train/loss,██▇▇▇▆▅▅▅▅▅▃▃▃▃▃▃▁▁▁▁▁▁▁▁▁▁▁
train/mean_token_accuracy,▁▂▂▂▂▃▄▃▃▄▄▅▆▆▆▆▆▇▇▇▇▇███████
train/num_tokens,▁▁▂▂▂▂▃▃▃▃▄▄▄▄▅▅▅▅▆▆▆▆▇▇▇▇███
total_flos,6.089261301578342e+16
train/epoch,4.98358
train/global_step,1405
train/grad_norm,0.39551


# Inference

In [2]:
max_new_tokens = 256
model_name = "Qwen/Qwen2.5-1.5B-Instruct"
checkpoint_path = 'finetune_qwen_results/Qwen2.5-1.5B-Instruct-salary/checkpoint-1405/'

In [3]:
model = AutoModelForCausalLM.from_pretrained(checkpoint_path, device_map="cuda:0")
tokenizer = AutoTokenizer.from_pretrained(
   model_name, padding=True, truncation=True
)

Sliding Window Attention is enabled but not implemented for `sdpa`; unexpected results may be encountered.


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [4]:
os.environ['HF_TOKEN'] = 'hf_...'
model.push_to_hub('qwen_salary')
tokenizer.push_to_hub('qwen_salary')

[2025-04-27 14:21:13,164] [INFO] [real_accelerator.py:158:get_accelerator] Setting ds_accelerator to cuda (auto detect)


model-00002-of-00002.safetensors:   0%|          | 0.00/1.18G [00:00<?, ?B/s]

Upload 2 LFS files:   0%|          | 0/2 [00:00<?, ?it/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

README.md:   0%|          | 0.00/5.17k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/11.4M [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/huwar/qwen_salary/commit/c056bbdef4d7d422d70e51c6447330405598190a', commit_message='Upload tokenizer', commit_description='', oid='c056bbdef4d7d422d70e51c6447330405598190a', pr_url=None, repo_url=RepoUrl('https://huggingface.co/huwar/qwen_salary', endpoint='https://huggingface.co', repo_type='model', repo_id='huwar/qwen_salary'), pr_revision=None, pr_num=None)

In [14]:
test_df = pd.read_csv('data/salary_labelled_test_set_cleaned.csv')

Example:

In [15]:
messages_static = [
    {"role": "system", "content": "You are an expert job ad annotator. Your task is to extract structured salary information from job descriptions in the format: min-max-currency-frequency. If salary is not found, return: 0-0-None-None."},
]

In [16]:
i = 0

desc = {
    "job_title": test_df.iloc[i].job_title,
    "job_ad_details": test_df.iloc[i].job_ad_details,
    "nation_short_desc": test_df.iloc[i].nation_short_desc,
    "salary_additional_text": test_df.iloc[i].salary_additional_text,
}
desc_str = str(desc)

messages = copy.deepcopy(messages_static)
messages.append({
    "role": "user",
    "content": (
        f"{desc_str} Extract structured salary information from this job descriptions in the format: min-max-currency-frequency. "
        "Respond in JSON: {\"MinSalary\": \"\", \"MaxSalary\": \"\", \"Currency\": \"\", \"Frequency\": \"\"}. "
        "If not provided explicitly, output 0 for \"MinSalary\" and \"MaxSalary\", and \"None\" for \"Currency\" and \"Frequency\". "
        "If the salary is mentioned, always output a range, where MinSalary and MaxSalary can be equal. "
        "Use 'nation_short_desc' to determine the correct currency. "
        "Output the currency as 3 letters. Use adverb to output frequency (annual, monthly, daily or hourly)."
    )
})

In [17]:
messages

[{'role': 'system',
  'content': 'You are an expert job ad annotator. Your task is to extract structured salary information from job descriptions in the format: min-max-currency-frequency. If salary is not found, return: 0-0-None-None.'},
 {'role': 'user',
  'content': '{\'job_title\': \'Cashier \', \'job_ad_details\': \'Bertanggungjawab sebagai cashierMengurus semua rekod mengenai cek yang diterimaMenyediakan laporan yang diperlukan oleh HQ (Jabatan Akaun dan Jabatan Sumber Manusia)Kiraan stok bulananSemua kerja lain yang ditetapkan oleh pengurus cawangan dan supervisor pada bila-bila mengikut keperluanKeperluanBerkelulusan SPM / O Level / SKM Level 1 / SKM Level 2 / SKM Level 3 atau setarafSedikit kemahiran tentang komputerMenepati masaKerja overtime (Jika diperlukan)Gaji RM 1500 – 1800++ Calon berminat boleh whatsapp 010-3938581Seng Li Marketing Sdn Bhd is a One-Stop Auto Parts Trading CompanySalary : RM 1500 – 1800\', \'nation_short_desc\': \'MY\', \'salary_additional_text\': \'RM\

In [18]:
prompt = tokenizer.apply_chat_template(messages, tokenize=False)

In [19]:
prompt

'<|im_start|>system\nYou are an expert job ad annotator. Your task is to extract structured salary information from job descriptions in the format: min-max-currency-frequency. If salary is not found, return: 0-0-None-None.<|im_end|>\n<|im_start|>user\n{\'job_title\': \'Cashier \', \'job_ad_details\': \'Bertanggungjawab sebagai cashierMengurus semua rekod mengenai cek yang diterimaMenyediakan laporan yang diperlukan oleh HQ (Jabatan Akaun dan Jabatan Sumber Manusia)Kiraan stok bulananSemua kerja lain yang ditetapkan oleh pengurus cawangan dan supervisor pada bila-bila mengikut keperluanKeperluanBerkelulusan SPM / O Level / SKM Level 1 / SKM Level 2 / SKM Level 3 atau setarafSedikit kemahiran tentang komputerMenepati masaKerja overtime (Jika diperlukan)Gaji RM 1500 – 1800++ Calon berminat boleh whatsapp 010-3938581Seng Li Marketing Sdn Bhd is a One-Stop Auto Parts Trading CompanySalary : RM 1500 – 1800\', \'nation_short_desc\': \'MY\', \'salary_additional_text\': \'RM\\xa01,500 – RM\\xa0

In [20]:
inputs = tokenizer(prompt, return_tensors="pt").to(model.device)
with torch.inference_mode():
    outputs = model.generate(**inputs, max_new_tokens=max_new_tokens, pad_token_id=tokenizer.eos_token_id)
outputs = outputs[:, inputs['input_ids'].shape[-1]:]
response = tokenizer.decode(outputs[0], skip_special_tokens=True)

In [21]:
response

'assistant\n{"MinSalary": "1500", "MaxSalary": "1800", "Currency": "MYR", "Frequency": "MONTHLY"}'

In [22]:
assistant_token = 'assistant\n'
response[response.find(assistant_token) + len(assistant_token):]

'{"MinSalary": "1500", "MaxSalary": "1800", "Currency": "MYR", "Frequency": "MONTHLY"}'

Test set:

In [23]:
# df to store model predictions
test_pred_df = pd.DataFrame(columns=["y_pred"])

In [24]:
messages_static = [
    {"role": "system", "content": "You are an expert job ad annotator. Your task is to extract structured salary information from job descriptions in the format: min-max-currency-frequency. If salary is not found, return: 0-0-None-None."},
]

In [25]:
for i in tqdm(range(len(test_df)), position=0, leave=True):
    desc = {
        "job_title": test_df.iloc[i].job_title,
        "job_ad_details": test_df.iloc[i].job_ad_details,
        "nation_short_desc": test_df.iloc[i].nation_short_desc,
        "salary_additional_text": test_df.iloc[i].salary_additional_text,
    }
    desc_str = str(desc)

    messages = copy.deepcopy(messages_static)
    messages.append({
        "role": "user",
        "content": (
            f"{desc_str} Extract structured salary information from this job descriptions in the format: min-max-currency-frequency. "
            "Respond in JSON: {\"MinSalary\": \"\", \"MaxSalary\": \"\", \"Currency\": \"\", \"Frequency\": \"\"}. "
            "If not provided explicitly, output 0 for \"MinSalary\" and \"MaxSalary\", and \"None\" for \"Currency\" and \"Frequency\". "
            "If the salary is mentioned, always output a range, where MinSalary and MaxSalary can be equal. "
            "Use 'nation_short_desc' to determine the correct currency. "
            "Output the currency as 3 letters. Use adverb to output frequency (annual, monthly, daily or hourly)."
        )
    })
    
    prompt = tokenizer.apply_chat_template(messages, tokenize=False)
    inputs = tokenizer(prompt, return_tensors="pt").to(model.device)
    with torch.inference_mode():
        outputs = model.generate(**inputs, max_new_tokens=max_new_tokens, pad_token_id=tokenizer.eos_token_id)
    outputs = outputs[:, inputs['input_ids'].shape[-1]:]
    response = tokenizer.decode(outputs[0], skip_special_tokens=True)
    
    assistant_token = 'assistant\n'
    answer_str = response[response.find(assistant_token) + len(assistant_token):]

    # format the output
    try:
        answer_str_ = answer_str[answer_str.find('{'):answer_str.find('}') + 1]
        answer_str_ = answer_str_.replace('“', '"')
        answer_str_ = answer_str_.replace('”', '"')
        answer = json.loads(answer_str_)

        label = f"{answer['MinSalary']}-{answer['MaxSalary']}-{answer['Currency'].upper()}-{answer['Frequency'].upper()}"

    except json.JSONDecodeError:
        print(f"Failed to parse model output as JSON: {answer_str}")
        label = "ERROR " + answer_str

    test_pred_df.loc[len(test_pred_df)] = label

100%|██████████| 567/567 [21:06<00:00,  2.23s/it]


In [26]:
# export the dataframe to a new csv file
test_pred_df.to_csv('salary_labelled_test_set_qwen_finetune_preds.csv', index=False)

# Metrics

In [1]:
import json
import string
import pandas as pd

In [2]:
def simple_post_process(row):
    if 'ERROR' not in row:
        row = row.translate(str.maketrans('', '', '"'))
        try:
            mn, mx, cur, freq = row.split('-')
        except Exception:
            row_split = row.split('-')
            mn = row_split[0]
            mx = ''.join(c for c in mn if c.isdigit())
            mx = row_split[-3]
            mx = ''.join(c for c in mx if c.isdigit())
            cur = row_split[-2]
            freq = row_split[-1]
        # 'NONE' -> 'None'
        cur = 'None' if cur == 'NONE' else cur
        freq = 'None' if freq == 'NONE' else freq
        # cast min and max salary to int
        try:
            mn = int(round(float(mn)))
        except Exception:
            mn = 0
        try:
            mx = int(round(float(mx)))
        except Exception:
            mx = 0
        return str(mn), str(mx), cur, freq, f'{mn}-{mx}-{cur}-{freq}'
    else:
        row = row.strip()
        try:
            row = row[row.find('{'):-4]
            row_data = json.loads(row)
        except Exception:
            return 0, 0, 'None', 'None', '0-0-None-None'
        row_data = row_data['Full Time'] if 'Full Time' in row_data else (row_data['Full-timer'] if 'Full-timer' in row_data else row_data)
        try:
            mn = row_data['MinSalary']
            mx = row_data['MaxSalary']
            cur = row_data['Currency']
            freq = row_data['Frequency']
        except Exception:
            return 0, 0, 'None', 'None', '0-0-None-None'
        # 'NONE' -> 'None'
        cur = 'None' if cur == 'NONE' else cur
        freq = 'None' if freq == 'NONE' else freq
        # cast min and max salary to int
        mn = int(round(float(mn)))
        mx = int(round(float(mx)))
        return str(mn), str(mx), cur, freq, f'{mn}-{mx}-{cur}-{freq}'

def split_target(row):
    mn, mx, cur, freq = row.split('-')
    return mn, mx, cur, freq

def get_accuracy(path_to_preds):
    preds = pd.read_csv(path_to_preds)
    test_df = pd.read_csv('data/salary_labelled_test_set_cleaned.csv')
    
    test_df['y_pred'] = preds.values.reshape(-1)
    
    test_df['min_salary_pred'], test_df['max_salary_pred'], test_df['currency_pred'], test_df['freq_pred'], test_df['y_pred'] = zip(*test_df['y_pred'].map(simple_post_process))
    test_df['min_salary_true'], test_df['max_salary_true'], test_df['currency_true'], test_df['freq_true'] = zip(*test_df['y_true'].map(split_target))
    
    acc_overall = (test_df['y_pred'] == test_df['y_true']).mean() * 100
    acc_min = (test_df['min_salary_pred'] == test_df['min_salary_true']).mean() * 100
    acc_max = (test_df['max_salary_pred'] == test_df['max_salary_true']).mean() * 100
    acc_curr = (test_df['currency_pred'] == test_df['currency_true']).mean() * 100
    acc_freq = (test_df['freq_pred'] == test_df['freq_true']).mean() * 100
    
    res = pd.DataFrame(
        {
            'Overall': round(acc_overall, 2),
            'Min Salary': round(acc_min, 2),
            'Max Salary': round(acc_max, 2),
            'Currency': round(acc_curr, 2),
            'Frequency': round(acc_freq, 2),
        },
        index=['Accuracy (%)']
    )
    
    return res

In [3]:
get_accuracy('salary_labelled_test_set_qwen_finetune_preds.csv')

,Overall,Min Salary,Max Salary,Currency,Frequency
Accuracy (%),94.0,95.59,95.94,97.88,98.06
